In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
        print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
traindir = '../input/chest-xray-pneumonia/chest_xray/train'
testdir = '../input/chest-xray-pneumonia/chest_xray/test'
valdir = '../input/chest-xray-pneumonia/chest_xray/val'

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import glob

train_p = glob.glob(traindir+'/PNEUMONIA/*jpeg')
train_n = glob.glob(traindir+'/NORMAL/*jpeg')

data = pd.DataFrame(np.concatenate([[0]*len(train_n), [1]*len(train_p)]), columns=["class"])



## Transforming Images 
- Resizing 
- Normalizing 
- Applying Random Rotations

In [6]:
from torchvision import transforms
train_trnsf = transforms.Compose([transforms.RandomRotation((-20,20)),
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_trnsf = transforms.Compose([transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [7]:
from torchvision.datasets import ImageFolder 
train_ds = ImageFolder(traindir, train_trnsf)
test_ds = ImageFolder(testdir, test_trnsf)
val_ds = ImageFolder(valdir, test_trnsf)

In [8]:
dataset = []
dataset = torch.utils.data.ConcatDataset([dataset, train_ds])
# dataset = torch.utils.data.ConcatDataset([dataset, test_ds])
# dataset = torch.utils.data.ConcatDataset([dataset, val_ds])

In [9]:
from torch.utils.data import DataLoader
batch_size = 64
trainloader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
testloader = DataLoader(test_ds, batch_size, shuffle = True, num_workers=2, pin_memory=True)
valloader = DataLoader(val_ds, batch_size*2, shuffle = True, num_workers=2, pin_memory=True)

In [10]:
from matplotlib import pyplot as plt
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs Number of epochs')

## Defining the ResNet model 

In [11]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        self.in_channels = intermediate_channels * 4


        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


# def ResNet50(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


# def ResNet101(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


# def ResNet152(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)

### Function for training and evaluating models with different parameters for optuna to search for the best one 

In [12]:
def train_and_evaluate(param, model):
    
    run_epochs(param, model)
        
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = (100*correct/total)
    
    return accuracy

### Defining the objective function that tries and checks the accuracy of the model with different learning rates and optimizers for finding the best hyperparameters

In [ ]:
def objective(trial):
    
    model = ResNet(block, [2,2,2,2], 3, 2)
    
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        'epochs': 2
        #'criterion': trial.suggest_categorical("criterion", ["NLLLoss", "CrossEntropyLoss", "GaussianNLLLoss"])
    }   
    
    accuracy = train_and_evaluate(params, model)
    
    return accuracy
    

### Executing the optuna study function that expermiments with different hyperparameters and finds the one with the maximum accuracy

In [ ]:
EPOCHS = 30

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

### Using the best hyperparameters found by optuna

In [15]:
params = {
    "optimizer": "RMSprop",
    "learning_rate": 0.0003264,
    "epochs": 8
}

### Function to train the model

In [13]:
def run_epochs(param,model,trainloader):
    model = model.to(device)
    best_model = model
    #criterion = getattr(nn, param['criterion'])()
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr=param['learning_rate'])
    num_epochs = param['epochs']
    
    step = 0
    train_losses = []
    accuracies = []
    best_acc = 0
    accuracy = 0

    torch.cuda.empty_cache()

    for epoch in range(num_epochs):
        running_accuracy = []
        losses = []
        for i, (images,labels) in enumerate(trainloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            losses.append(loss.item())            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predictions = outputs.max(1)
            num_correct = (predictions == labels).sum()
            running_train_acc = float(num_correct)/float(images.shape[0])
            running_accuracy.append(running_train_acc)
        
        train_losses.append(sum(losses) / len(losses))
        accuracy = sum(running_accuracy) / len(running_accuracy)
        accuracies.append(accuracy)
        print("Accuracy for epoch {} = {}".format(epoch+1, accuracy*100))
                
#             train_acc += running_train_acc
#             train_loss += loss.item()
#             avg_train_acc = train_acc / len(trainloader)
#             avg_train_loss = train_loss / len(trainloader)

            
    return train_losses, accuracies, model

### Function to Check the model's accuracy with other data

In [14]:
def check_model(model, trainloader):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            

        accuracy = (100*correct/total)
    return accuracy

## Training the model

In [16]:
import time
start_time = time.time()

model = ResNet(block, [2,2,2,2], 3, 2)

losses, accuracies, model = run_epochs(params,model,trainloader)

print("--- %s minutes ---" % ((time.time() - start_time)/60))

### Plotting the losses

In [17]:
def plot_losses(losses):
    plt.plot(losses, '-bx')
    plt.xlabel('epoch')
    plt.ylabel('loss')

In [19]:
plot_losses(losses)

### Plotting the accuracies

In [18]:
def plot_accuracies(accuracy):
    plt.plot(accuracies,'-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs Number of Epochs')

In [20]:
plot_accuracies(accuracies)

In [23]:
print('Validation accuracy = %f' % check_model(model,valloader))

In [21]:
print('Test accuracy = %f' % check_model(model, testloader))

In [24]:
torch.save(model.state_dict(), 'model.pth')

## Dividing the dataset into 5 folds for using the K fold Cross Validation

In [ ]:
# Configuration options
k_folds = 5
epochs = 2
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
models = {}
c = 1

## K-fold Cross Validation model evaluation

In [ ]:
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)

    # Init the neural network
    model = ResNet(block, [2,2,2,2], 3, 2)

    # Run the training loop for defined number of epochs
    losses, accuracies, accuracy = run_epochs(params,model)

    # Process is complete.
    print('Training process has finished. Saving trained model.')
    print('--------------------------------')
    models['model'+str(c)] = model
    c += 1

In [ ]:
from sklearn.metrics import accuracy_score

## Function for predicting the output using the average of the outputs from the 5 trained models, thus using and applying the ensembling technique

In [ ]:
def predict(models, data):
    with torch.no_grad():
        correct = 0
        total = 0
        out = 0


        
        for images,labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            prediction = []
            for i in range(5):
                outputs = models['model'+str(i+1)](images)
                _, predicted = torch.max(outputs.data, 1)
                prediction += predicted 
            
            prediction = [x/5 for x in prediction]
            prediction = [(1 if i>0.5 else 0) for i in prediction]
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = (100*correct/total)
    
    return accuracy

In [ ]:
accuracy = predict(models,dataset)

In [ ]:
print('Accuracy = %f' % accuracy)

### Code for saving the models

In [ ]:
for i in range(5):
    torch.save(models['model'+str(i+1)].state_dict(), f'model{i+1}.pth')

### Code for loading the models

In [ ]:
for i in range(5):
    torch.load(models['model'+str(i+1)].state_dict(), f'model{i+1}.pth')